In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


### Load Model

In [2]:
model = load_model("Predictor_model.h5")

In [3]:
#Function to get emojis
def get_emojis():
    emojis_folder = "Emojis/"
    emojis = []
    
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+".png", -1))
    return emojis

In [4]:
#Function for predicting emojis 
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_prob = model.predict(processed)[0]
    pred_class = list(pred_prob).index(max(pred_prob))
    return max(pred_prob),pred_class

#Function for processing image
def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x,image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1,image_x,image_y,1))
    return img
    

In [5]:
#Functions for overlaying emojis
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w,h))
    try:
        image[y:y + h, x:x + w] = blend_transparent(image[y:y + h, x:x + w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    overlay_img = overlay_t_img[:, :, :3]
    overlay_mask = overlay_t_img[:, :, 3:]
    
    background_mask = 255 - overlay_mask
    
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)
    
    face_part = (face_img * (1/255.0)) * (background_mask * (1/255.0))
    overlay_part = (overlay_img * (1/255.0)) * (overlay_mask * (1/255.0))
    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

### Experimentation

In [6]:
emojis = get_emojis()

cap = cv2.VideoCapture(0)
x, y, w, h = 300, 50, 350, 350

while(cap.isOpened()):
        ret, img = cap.read()
        #Flip the read frame to get correct orientation
        img = cv2.flip(img, 1)
        #Convert to HSV Format
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        #THRESH SCREEN
        #Set mask in range of skin color
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        #Convert to grey scale
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        #Initialize kernel
        kernel_square = np.ones((5, 5), np.uint8)
        #Improve the quality of captured gesture
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)

        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
        #Set thresh according to particular area
        thresh = thresh[y:y + h, x:x + w]
        #Find cotours in thresh
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
    
        #IF contours exixts
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            #Condition for starting the process of capturing gesture
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                new_img = thresh[y:y + h1, x:x + w1]
                new_img = cv2.resize(new_img, (50, 50))
                #Get probabilities and predictions
                pred_prob, pred_class = keras_predict(model, new_img)
                print(pred_class, pred_prob)
                #Get emojis on the frame
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350 
        #FRAME SCREEN Heading
        cv2.imshow("Frame", img)
        #THRESH SCREEN Heading
        cv2.imshow("Contours", thresh)
        
        
        keypress = cv2.waitKey(15)
        if keypress == 27:
            break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 0.9949608
12 0.9949608
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
15 1.0
15 1.0
12 1.0
12 1.0
15 1.0
15 1.0
12 1.0
12 1.0
12 1.0
15 1.0
15 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 0.99999976
15 1.0
15 1.0
12 1.0
12 1.0
12 1.0
15 1.0
15 1.0
14 1.0
14 1.0
12 0.99981683
12 0.99981683
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
14 1.0
14 1.0
14 1.0
14 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
14 1.0
14 1.0
14 1.0
14 1.0
15 0.99996614
15 0.99996614
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
14 1.0
1 1.0
14 1.0
14 1.0
12 1.0
12 1.0
14 1.0
14 

KeyboardInterrupt: 